In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/layoffs-2022/layoffs.csv


In [2]:
df = pd.read_csv('/kaggle/input/layoffs-2022/layoffs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              2225 non-null   object 
 1   location             2225 non-null   object 
 2   industry             2224 non-null   object 
 3   total_laid_off       1530 non-null   float64
 4   percentage_laid_off  1498 non-null   float64
 5   date                 2225 non-null   object 
 6   stage                2219 non-null   object 
 7   country              2225 non-null   object 
 8   funds_raised         2030 non-null   float64
dtypes: float64(3), object(6)
memory usage: 156.6+ KB


In [3]:
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,TikTok India,Mumbai,Consumer,40.0,NaN,2023-02-10,Acquired,India,9400.0
1,Rigetti Computing,SF Bay Area,Hardware,NaN,0.28,2023-02-10,Post-IPO,United States,298.0
2,Yahoo,SF Bay Area,Consumer,1600.0,0.20,2023-02-09,Acquired,United States,6.0
3,Deliveroo,London,Food,350.0,NaN,2023-02-09,Post-IPO,United Kingdom,1700.0
4,Olive AI,Columbus,Healthcare,215.0,0.35,2023-02-09,Series H,United States,856.0


In [4]:
# top 10 countries which the layoffs companies in
df.country.value_counts()[:10]

United States     1456
India              140
Canada              99
United Kingdom      71
Brazil              70
Germany             65
Israel              52
Australia           47
Singapore           32
Indonesia           23
Name: country, dtype: int64

In [5]:
# keep only companies in US
df = df[df.country == 'United States']
df.shape

(1456, 9)

In [6]:
# check NaN values
df.isnull().sum()

company                  0
location                 0
industry                 1
total_laid_off         485
percentage_laid_off    475
date                     0
stage                    4
country                  0
funds_raised           108
dtype: int64

In [7]:
# drop rows missing industry and stage data
df = df[df.industry.notna() & df.stage.notna()]
df.shape

(1451, 9)